In [1]:
lines = sc.textFile('./data/uid_iid')

In [2]:
# target, etime, subset = "12-19-0", "12-18-23", {}
target, etime, subset = "12-18-0", "12-17-23", {}
# target, etime, subset = "12-17-0", "12-16-23", {}
# target, etime, subset = "12-16-0", "12-15-23", {}
# target, etime, subset = "12-15-0", "12-14-23", {}
# target, etime, subset = "12-14-0", "12-13-23", {}
# target, etime, subset = "12-13-0", "12-12-23", {}
# target, etime, subset = "12-12-0", "12-11-23", {}
# target, etime, subset = "12-11-0", "12-10-23", {}
# target, etime, subset = "12-10-0", "12-09-23", {}
# target, etime, subset = "12-09-0", "12-08-23", {}
# target, etime, subset = "12-08-0", "12-07-23", {}
# target, etime, subset = "12-07-0", "12-06-23", {}
# target, etime, subset = "12-06-0", "12-05-23", {}
# target, etime, subset = "12-05-0", "12-04-23", {}
# target, etime, subset = "12-04-0", "12-03-23", {}
# target, etime, subset = "12-03-0", "12-04-23", {}
# target, etime, subset = "12-02-0", "12-01-23", {}
# target, etime, subset = "12-01-0", "11-30-23", {}

In [3]:
import fileinput

In [4]:
for line in fileinput.input("./data/tianchi_fresh_comp_train_item.csv"):
    subset[line.strip().split(",")[0]] = True

In [5]:
def extract(line):
    import time
    import itertools
    (uid, iid, ict) = line.strip().split("\t")[0].split(" ")
    if subset.has_key(iid):
        items = filter(lambda x:x[0]>0, [(int(time.mktime(time.strptime('2014-'+etime,'%Y-%m-%d-%H'))-time.mktime(time.strptime('2014-'+i.split(",")[0],'%Y-%m-%d-%H')))/(24*3600)+1, int(i.split(",")[1])) for i in line.strip().split("\t")[1].split(" ")])
        f, inf = [0]*22, 100
        buy = filter(lambda x:x[1]==4, items)
        last = buy[-1][0] if len(buy)!=0 else inf
        a1 = filter(lambda x:x[0]<last and x[1]==1, items) 
        a2 = filter(lambda x:x[0]<last and x[1]==2, items) 
        a3 = filter(lambda x:x[0]<last and x[1]==3, items)
        # statistics
        f[0] = a2[-1][0] if len(a2)!=0 else inf # number of days till last collect after buy
        f[1] = a3[-1][0] if len(a3)!=0 else inf # number of days till last cart after buy
        f[2] = len(a1) # number of click after buy
        f[3] = len(a2) # number of collect after buy
        f[4] = len(a3) # number of cart after buy
        f[5] = len(filter(lambda x:x[0]<=1, items)) # number of interaction last day
        f[6] = len(filter(lambda x:x[0]==2, items)) # number of interaction penultimate day
        f[7] = len(filter(lambda x:x[0]==3, items)) # number of interaction third last day
        f[8] = len(filter(lambda x:x[0]==4, items)) # number of interaction fourth last day
        f[9] = len(filter(lambda x:x[0]<=7, items)) # number of interaction last week
        f[10] = len(buy) # number of buy in history
        f[11] = last # number of days till last buy
        f[12] = len(set([item[0] for item in items if item[0]<=3])) # number of days of interaction last three days
        f[13] = len(set([item[0] for item in items if item[0]<=7])) # number of days of interaction last week
        f[14] = len(set([item[0] for item in items if item[0]<=21])) # number of days of interaction last three weeks
        f[15] = items[-1][0] if len(items)!=0 else inf # number of days till last interaction
        inter = [len(list(i)) for _,i in itertools.groupby(items, lambda x: x[0])]
        f[16] = len(inter) # number of days interaction
        f[17] = max(inter) if len(inter)!=0 else 0 # most number of interactions one day
        f[18] = len(filter(lambda x:x[1]==1, items)) # total number of interactions
        f[19] = len(filter(lambda x:x[1]==2, items)) # total number of collect
        f[20] = len(filter(lambda x:x[1]==3, items)) # total number of cart
        f[21] = items[0][0]-items[-1][0]+1 if len(items)!=0 else 0 # duration from firts interaction till last interaction
        return (uid+"\t"+iid+"\t"+ict+"\t"+"\t".join([str(i) for i in f]))
    else:
        return ("")

In [6]:
counts = lines.map(lambda x : extract(x))\
                .filter(lambda x : x!="")

In [7]:
output = counts.saveAsTextFile("./feature/"+target+"/user_item/")

Traceback (most recent call last):
  File "/tmp/kernel-PySpark-e38eed94-64e1-4487-a6d3-03577ef8f20e/pyspark_runner.py", line 205, in <module>
    output.reset()
AttributeError: 'NoneType' object has no attribute 'reset'


In [8]:
# counts.count()

Name: org.apache.toree.interpreter.broker.BrokerException
Message: null was reset!
StackTrace: org.apache.toree.interpreter.broker.BrokerState$$anonfun$reset$1.apply(BrokerState.scala:191)
org.apache.toree.interpreter.broker.BrokerState$$anonfun$reset$1.apply(BrokerState.scala:189)
scala.collection.Iterator$class.foreach(Iterator.scala:893)
scala.collection.AbstractIterator.foreach(Iterator.scala:1336)
scala.collection.IterableLike$class.foreach(IterableLike.scala:72)
scala.collection.AbstractIterable.foreach(Iterable.scala:54)
org.apache.toree.interpreter.broker.BrokerState.reset(BrokerState.scala:189)
org.apache.toree.kernel.interpreter.pyspark.PySparkService$$anonfun$pySparkProcess$2.apply(PySparkService.scala:63)
org.apache.toree.kernel.interpreter.pyspark.PySparkService$$anonfun$pySparkProcess$2.apply(PySparkService.scala:61)
org.apache.toree.interpreter.broker.BrokerProcessHandler.onProcessComplete(BrokerProcessHandler.scala:67)
org.apache.commons.exec.DefaultExecutor$1.run(Defau

In [ ]:
# counts.take(20)